In [1]:
pip install tensorflow_text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

Import the dataset (Dataset is taken from kaggle)

In [3]:
df = pd.read_csv("spam.csv")

In [4]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

15% spam emails, 85% ham emails: This indicates class imbalance

In [6]:
df_spam = df[df['Category']=='spam']

In [7]:
df_ham = df[df['Category']=='ham']

In [8]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])

In [9]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])

In [10]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [11]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [12]:
df_balanced.sample(5)

,Category,Message,spam
3809,ham,Mm you ask him to come its enough :-),0
4280,spam,URGENT. Important information for 02 user. Tod...,1
508,ham,You should know now. So how's anthony. Are you...,0
3073,ham,Ok lor... But buy wat?,0
904,ham,Lovely smell on this bus and it ain't tobacco...,0


Split it into training and test data set


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

Now lets import BERT model and get embeding vectors for few sample statements



In [15]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Build Model

In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [17]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [18]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [31]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS,
)

Train the model


In [27]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 12s 340ms/step - loss: 0.2184 - accuracy: 0.9286 - precision: 0.9181 - recall: 0.9411
Epoch 2/10
35/35 [==============================] - 12s 348ms/step - loss: 0.2107 - accuracy: 0.9339 - precision: 0.9339 - recall: 0.9339
Epoch 3/10
35/35 [==============================] - 12s 353ms/step - loss: 0.2083 - accuracy: 0.9366 - precision: 0.9297 - recall: 0.9446
Epoch 4/10
35/35 [==============================] - 12s 349ms/step - loss: 0.2112 - accuracy: 0.9312 - precision: 0.9305 - recall: 0.9321
Epoch 5/10
35/35 [==============================] - 12s 349ms/step - loss: 0.2002 - accuracy: 0.9330 - precision: 0.9292 - recall: 0.9375
Epoch 6/10
35/35 [==============================] - 12s 342ms/step - loss: 0.2006 - accuracy: 0.9304 - precision: 0.9304 - recall: 0.9304
Epoch 7/10
35/35 [==============================] - 12s 344ms/step - loss: 0.1963 - accuracy: 0.9384 - precision: 0.9361 - recall: 0.9411
Epoch 8/10
35/35 [================

Evaluate model

In [22]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

12/12 [==============================] - 4s 331ms/step


In [23]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,

Making predictions

In [24]:
sample_dataset = [
 'You can win a lot of money, register in the link below.'
 'You have an iPhone 10, spin the image below to claim your prize and it will be delivered in your door step',
 'You have an offer, the company will give you 50% off on every item purchased.',
 "See you monday, we have alot to talk about the future of this company ."
]

In [25]:
model.predict(sample_dataset)

1/1 [==============================] - 1s 784ms/step


array([[0.9162448 ],
       [0.48993325],
       [0.12126531]], dtype=float32)

From the output above, the first three email messages have been classified as spam. They have a prediction probability that is greater than 0.5. The last two email messages have been classified as ham. They have a prediction probability that is less than 0.5. These are the right predictions and show we have successfully built our text classification model.